In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential

# Define the input image size and batch size
img_width, img_height = 224, 224
batch_size = 32

# Define the directory for the training data
train_data_dir = '/content/drive/MyDrive/AgeClassification/train/'

# Define the directories for the test data
test_data_dir = '/content/drive/MyDrive/AgeClassification/test/'

# Create an ImageDataGenerator for the training and validation sets with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,  # Use 20% of the training data for validation
)

# Create an ImageDataGenerator for the test set
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the training and validation images from the directory
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    classes=['children', 'adults'],
    shuffle=True,
    subset='training',  # Use the subset of the data for training
)

val_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    classes=['children', 'adults'],
    shuffle=False,
    subset='validation',  # Use the subset of the data for validation
)

# Load the test images from the directory
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    classes=['children', 'adults'],
    shuffle=False,
)

# Define the CNN model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model with binary crossentropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model on the training data and validate on the validation data
model.fit(train_generator, epochs=10, validation_data=val_generator)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator, verbose=2)

# Print the test accuracy
print('Test accuracy:', test_acc)

Found 553 images belonging to 2 classes.
Found 138 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
Epoch 1/10
18/18 [==============================] - 119s 6s/step - loss: 0.7021 - accuracy: 0.5244 - val_loss: 0.6929 - val_accuracy: 0.5145
Epoch 2/10
18/18 [==============================] - 92s 5s/step - loss: 0.6906 - accuracy: 0.5262 - val_loss: 0.6815 - val_accuracy: 0.5942
Epoch 3/10
18/18 [==============================] - 88s 5s/step - loss: 0.6743 - accuracy: 0.5461 - val_loss: 0.6852 - val_accuracy: 0.5652
Epoch 4/10
18/18 [==============================] - 86s 5s/step - loss: 0.6625 - accuracy: 0.5913 - val_loss: 0.7575 - val_accuracy: 0.5797
Epoch 5/10
18/18 [==============================] - 88s 5s/step - loss: 0.6544 - accuracy: 0.6166 - val_loss: 0.7602 - val_accuracy: 0.5870
Epoch 6/10
18/18 [==============================] - 88s 5s/step - loss: 0.6563 - accuracy: 0.6076 - val_loss: 0.7372 - val_accuracy: 0.6014
Epoch 7/10
18/18 [==================

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Define the input image size and batch size
img_width, img_height = 224, 224
batch_size = 32

# Define the directory for the training data
train_data_dir = '/path/to/training/folder'

# Create an ImageDataGenerator for the training and validation sets with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,  # Use 20% of the training data for validation
)

# Load the training and validation images from the directory
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    classes=['children', 'adults'],
    shuffle=True,
    subset='training',  # Use the subset of the data for training
)

val_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    classes=['children', 'adults'],
    shuffle=False,
    subset='validation',  # Use the subset of the data for validation
)

# Load the test images from the directory
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    classes=['children', 'adults'],
    shuffle=False,
)

# Load the pre-trained ResNet50 model and remove the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Add a global average pooling layer and a dense output layer with sigmoid activation
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1, activation='sigmoid')(x)

# Define the full model by combining the base model and the added layers
model = Model(inputs=base_model.input, outputs=x)

# Freeze the layers of the pre-trained ResNet50 model to prevent retraining
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with binary crossentropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model on the training data and validate on the validation data
model.fit(train_generator, epochs=10, validation_data=val_generator)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator, verbose=2)

# Print the test accuracy
print('Test accuracy:', test_acc)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Define the input image size and batch size
img_width, img_height = 224, 224
batch_size = 32

# Define the directories for the training and validation data
train_data_dir = '/content/drive/MyDrive/AgeClassification/train/'
validation_data_dir = '/content/drive/MyDrive/AgeClassification/train/'
test_data_dir = '/content/drive/MyDrive/AgeClassification/train/'

# Create an ImageDataGenerator for the training set with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

# Create an ImageDataGenerator for the validation set
val_datagen = ImageDataGenerator(rescale=1./255)

# Create an ImageDataGenerator for the test set
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the training images from the directories
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    classes=['children', 'adults'],
    shuffle=True,
)

# Load the validation images from the directories
val_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    classes=['children', 'adults'],
    shuffle=False,
)

# Load the test images from the directories
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    classes=['children', 'adults'],
    shuffle=False,
)

# Load the pre-trained ResNet50 model and remove the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Add a global average pooling layer and a dense output layer with sigmoid activation
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1, activation='sigmoid')(x)

# Define the full model by combining the base model and the added layers
model = Model(inputs=base_model.input, outputs=x)

# Freeze the layers of the pre-trained ResNet50 model to prevent retraining
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with binary crossentropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model on the training data and validate on the validation data
model.fit(train_generator, epochs=10, validation_data=val_generator)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator, verbose=2)

# Print the test accuracy
print('Test accuracy:', test_acc)

Found 690 images belonging to 2 classes.
Found 690 images belonging to 2 classes.
Found 690 images belonging to 2 classes.
94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/10
11/22 [==============>...............] - ETA: 1:15 - loss: 0.7403 - accuracy: 0.5341

KeyboardInterrupt: ignored